### Spatial Thinking

## Part 3 of 3
# ...

## Reminder
<a href="#/slide-2-0" class="navigate-right" style="background-color:blue;color:white;padding:8px;margin:2px;font-weight:bold;">Continue with the lesson</a>

<br>
</br>
<font size="+1">

By continuing with this lesson you are granting your permission to take part in this research study for the Hour of Cyberinfrastructure: Developing Cyber Literacy for GIScience project. In this study, you will be learning about cyberinfrastructure and related concepts using a web-based platform that will take approximately one hour per lesson. Participation in this study is voluntary.

Participants in this research must be 18 years or older. If you are under the age of 18 then please exit this webpage or navigate to another website such as the Hour of Code at https://hourofcode.com, which is designed for K-12 students.

If you are not interested in participating please exit the browser or navigate to this website: http://www.umn.edu. Your participation is voluntary and you are free to stop the lesson at any time.

For the full description please navigate to this website: <a href="../../gateway-lesson/gateway/gateway-1.ipynb">Gateway Lesson Research Study Permission</a>.

</font>

In [ ]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout
import warnings
import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
import hourofci


# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

## Land-Use Suitability for Conservation Reserve  Program

Let's look at a Land Use Suitability example taken from <a href="https://link.springer.com/article/10.1007/s00477-018-1535-z">Şalap-Ayça and Jankowski (2008)</a>. 

In the United States, the Conservation Reserve Program (CRP) executes land use evaluation and <b>improves conservation by renting highly erodible cropland</b> or <b>other environmentally sensitive acreage</b> from farmers and ensuring plantation of species that <b>will improve environmental quality</b>.

CRP is achieved by a score based system called <b>Environmental Benefit Index</b>. It is based on five environmental factors, which are <b>wildlife,water quality, erosion, enduring benefits, and air quality</b>; and every factor with slightly different levels of preferences. 


<table>
    <tr style="background: #fff; text-align: left; vertical-align:">
        <td style="background: #fff; text-align: left; font-size: 20px;">
So decision makers come up with different <b>preferences</b> for each criteria and they are combined into a single overall score. 

When the <b>overall score</b> has higher ranks for the predetermined threshold value for the enrollment period, the land unit is left uncultivated for that term. 
        </td>
        <td style="width: 70%; background: #fff; text-align: left; vertical-align: top;"> <img src='supplementary/ebi.png' alt='map'>
        </td>
   </tr>
</table>



Even though the benefits of CRP have been observed for over 30 years, unfortunately due to fiscal limitations, only the most qualified land units are selected for incentive payments. 

Therefore, the overall objective becomes to define the most qualified areas that will maximize the conservation benefits by considering the multiple factors together with their varying preferences; which makes the EBI a typical example of a multi-criteria decision-making problem. 

Let's see how we can apply our generic recipe steps on this real case study.

Şalap-Ayça, S., Jankowski, P. Analysis of the influence of parameter and scale uncertainties on a local multi-criteria land use evaluation model. Stoch Environ Res Risk Assess 32, 2699–2719 (2018). https://doi.org/10.1007/s00477-018-1535-z.



### How many criteria do we have?

In [ ]:
widget1 = widgets.RadioButtons(
    options = [2,5,7],
    description = '', style={'description_width': 'initial'},
    layout = Layout(width='100%',display="flex", justify_content="flex-start"),
    value = None
)

display(widget1)
def out():
    return print('There are five criteria in our problem!')

hourofci.SubmitBtn2(widget1, out)


### What are the alternatives in this problem?
Answer : Parcels of Land / Agricultural Land units


In [ ]:
widget2 = widgets.RadioButtons(
    options = ['Option1','Parcels of Land / Agricultural Land units','Option3'],
    description = '', style={'description_width': 'initial'},
    layout = Layout(width='100%',display="flex", justify_content="flex-start"),
    value = None
)

display(widget2)
def out():
    return print('The answer is Parcels of Land / Agricultural Land units')

hourofci.SubmitBtn2(widget2, out)


### What is the objective?



In [ ]:
widget3 = widgets.RadioButtons(
    options = ['The higher the EBI score, the higher the level of acceptance of the land unit.','Option2','Option3'],
    description = '', style={'description_width': 'initial'},
    layout = Layout(width='100%',display="flex", justify_content="flex-start"),
    value = None
)

display(widget3)
def out():
    return print('The higher the EBI score, the higher the level of acceptance of the land unit.')

hourofci.SubmitBtn2(widget3, out)


### Recipe Step 1 : Define the set of evaluation criteria (map layers)
Evaluation Criteria are:

<ol>
    <li>
Wildlife
    </li>
    <li>
Water Quality
    </li>
    <li>
Erosion
    </li>
    <li>
Enduring Benefits
    </li>
    <li>
Air quality
    </li>
</ol>

And for this example, let’s assume that all these layers are given as an ASCII file for us. Let’s visualize them and see what they look like.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nArray = np.loadtxt('supplementary/n1.txt', skiprows=6)

plt.title('Wildlife')

plt.imshow(nArray)
plt.show()

Can you change the file name to n5.txt so that you can check the Air Quality layer?

In [ ]:
nArray = np.loadtxt('supplementary/n5.txt', skiprows=6)

plt.title('Air Quality')

plt.imshow(nArray)
plt.show()

Great! So we have all our criteria layers. Let’s move to the next step.

## Recipe Step 2 : Standardize each criterion map layer
In order to understand what standardization is and why we need it, let's check the maximum and minimum values of each layer.

For a single criterion notice how we can get minimum and maximum. Also, we have some no data values in this raster which is assigned as -9999. We are using the numpy mask function to eliminate -9999 becoming the minimum value.


In [ ]:
n1Array = np.loadtxt("supplementary/n1.txt", skiprows=6)
masked_n1Array = np.ma.array(n1Array, mask=(n1Array==-9999)) 

n1max = masked_n1Array.max() 
n1min = masked_n1Array.min()

print("Maximum of Wildlife criterion is :" +str(n1max)+ " and minimum of Wildlife criterion is : "+str(n1min))


Can you write a loop so that you can get all the minimum and maximum values from 5 criteria?

We have done it for you in the next slide!

In [ ]:
for i in range(1, 6):
    nArray = np.loadtxt("supplementary/n"+str(i)+".txt", skiprows=6) 
    masked_nArray = np.ma.array(nArray, mask=(nArray==-9999)) 
    nmax = masked_nArray.max() 
    nmin = masked_nArray.min() 
    print("Maximum of "+str(i)+"th criterion is :" +str(nmax)+ " and minimum of "+str(i)+"th criterion is : "+str(nmin))


Nice job!  Now as you have noticed we have various maxima. Namely when we say the maximum wildlife for a pixel, it is not the same value for air quality. After all, we can't add apples and oranges right? The criteria need to be standardized in order to make a further addition operation. 

The standardization procedure involves <b>transforming the raw data</b> to <b>standardized scores</b>. First, the maximum and minimum value from the raw data layer needs to be determined (luckily we already did that!). Then following rules apply:

If we need to <b>minimize</b> the criterion  -  we take the difference between <b>maximum value</b> and the <b>cell value</b> and divide it by the <b>range</b> (difference between minimum and maximum).

$$
standardized value = \frac{maximum- cell value}{range}
$$

If we need to <b>maximize</b> the criterion  -  we take the difference between the <b>cell value</b> and <b>minimum value</b> and divide it by the <b>range</b> (difference between minimum and maximum).

$$
standardized value = \frac{cell value-minimum}{range}
$$


The major advantage of this procedure is that the scale of measurement varies precisely between 0 and 1 for each criterion. <b>The worst standardized score is always equal to 0, and the best score equals 1.0.</b>

In [ ]:
n1Array = np.loadtxt("supplementary/n1.txt", skiprows=6)

masked_n1Array = np.ma.array(n1Array,mask=(n1Array==-9999)) 
n1max = masked_n1Array.max() 
n1min = masked_n1Array.min()

nrange = n1max- n1min 
standardizedn1Array = (masked_n1Array - n1min)/nrange

nstdmax = standardizedn1Array.max()
nstdmin = standardizedn1Array.min()

print("Standardized max is: " +str(nstdmax)+ " and standardized min is: "+str(nstdmin))


Can you write a loop so that you can standardize and save all standardized criteria values?

We have done it for you in the next slide!

In [ ]:
for i in range(1, 6):
    nArray = np.loadtxt("supplementary/n"+str(i)+".txt", skiprows=6)
    masked_nArray = np.ma.array(nArray, mask=(nArray==-9999))
    nmax = masked_nArray.max() 
    nmin = masked_nArray.min()

    nrange = nmax- nmin 
    standardizednArray = (masked_nArray - nmin)/nrange
    nstdmax = standardizednArray.max() 
    nstdmin = standardizednArray.min()
    print("Standardized max is : " +str(nstdmax)+ " and standardized min is : "+str(nstdmin)) 
    np. savetxt("supplementary/stdn"+str(i)+".txt", standardizednArray, fmt='%1.3f', delimiter=' ')


Well done! Now we are ready for the next step.

### Recipe Step 3 : Define the criterion weights (a.k.a relative importance) assigned to each criterion map
Criterion weights tell us how relatively important one criterion is over the other. For example, if the decision makers consensus that the protecting wildlife is the most important among all, then that criterion takes the highest weight. Criteria weights range from 0 to 1 and no matter how we distribute our preference among criteria the weight sum always equals to 1. And this is because there is always a trade off between criteria. If you are preferring something over another, then the rate of the latter one would be inversely proportional.

There are various consensus building tools to decide the weights (e.g. Analytical Hierarchy Process, Fuzzy Aggregate, Ordered Weighted Average, Outranking/Concordance Methods) however for this example we will proceed with the following weights for each criterion:

<center><img src='supplementary/criteriatable.png' width="300" height="700" alt='map'>


By looking at these weights, we can tell Wildlife , Water Quality, and Air Quality are considered slightly more important than Erosion and Enduring Benefits. Let’s see how we can reflect our preferences.

### Recipe Step 4: Construct the weighted standardized map layers - which means multiplying standardized map layers with their corresponding weights
If you remember, we generated standardized layers and saved them. Now, it is time to construct the weighted standardized map layers. In Multi-Criteria Decision Making literature this method is referred to as <b>Simple Additive Method</b> or <b>Weighted Linear Combination</b>. 
One advantage of WLC is that the method can easily be implemented within the GIS environment using map algebra operations. And the formula is simple too!

$$
Weighted \ standardized \ layer = criterion \ weight \times criterion \ value
$$

Let’s see how we can construct weighted standardized layers.


In [ ]:
W1 = 0.22 
W2 = 0.22 
W3 = 0.17 
W4 = 0.17 
W5 = 0.22
C1 = np.loadtxt("supplementary/stdn1.txt") 
C1 = np.ma.array(C1, mask=(C1==-9999)) 
weighted_std = W1*C1

Now it’s your turn to write a loop so that you can construct all weighted standardized criteria map layers.

In [ ]:
weight_list = [0.22, 0.22, 0.17, 0.17, 0.22]
for i in range(1, len(weight_list)+1):
    C = np.loadtxt("supplementary/stdn"+str(i)+".txt") 
    C = np.ma.array(C, mask=(C==-9999)) 
    weighted_std= weight_list[i-1]*C


### Recipe Step 5: Generate the overall score for each alternative using add overlay operation on the weighted standardized map layers 

Now it is time to calculate overall scores.  Namely, we need to overlay and see where we have the maximum of all criteria. This operation is another basic algebraic operation - sum! 

$$
Overall \ Score = \sum_{}{weight \times criteria \ value}
$$

∑ is a sum operator in mathematics. That means, we need to first multiply criterion value with criterion weight and repeat this for all criteria. Once we have the weighted layers, we need to sum them to get the total score.

In the next slide see how we can sum all the weighted standardized layers to get an overall score.


In [ ]:
weight_list = [0.22, 0.22, 0.17, 0.17, 0.22] 
overall = np.empty((1314,1308))
for i in range(1, len(weight_list)+1):
    C = np.loadtxt("supplementary/stdn"+str(i)+".txt") 
    C = np.ma.array(C, mask=(C==-9999)) 
    weighted_std= weight_list[i-1]*C
    overall= weighted_std +overall
plt.title("Overall Score")
plt.imshow(overall)
plt.show()

### Recipe Step 6: Rank the alternatives according to their overall score - the highest score is the best alternative
Final map seems promising but how are we going to decide which is best? Usually a ranking method is performed to get the optimum. Here since we don’t have a specific number of cells / size of land units to select, we will just rank the cell values so that we can visualize where we have higher score clusters.


In [ ]:
from scipy.stats import rankdata
ranked_overall = rankdata(overall, method='dense').reshape(overall.shape)
plt.title("Overall Ranked")
plt.imshow(ranked_overall)
plt.show()
print(" The best cell is located at row: " +str(np.argmax(np.max(overall, axis=1)))+ " and column" +str(np.argmax(np.max(overall, axis=0))))


### Recipe Step 7: If you are satisfied, well-done! 
If this analysis were performed on a vector layer, the output might have been a single feature. In a raster cell you might be interested in multiple cells. In order to perform such analysis you should start your exploratory analysis by observing the best scoring cells.  Here, we can see the highest scoring cells are located in the right top corner of our study area so that we can focus on the agricultural units located there next EBI period.


In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

ax = plt.subplot() 
im = ax.imshow(ranked_overall/(1314*1308))

plt.colorbar(im)
plt.show()


Congratulations!

<font style="color:red">
    A final statement is needed here to wrap up the lesson
    </font>

In [ ]:

# This code cell loads the Interact Textbox that will ask users for their name
# Once they click "Create Certificate" then it will add their name to the certificate template
# And present them a PDF certificate
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

from ipywidgets import interact

def make_cert(learner_name, lesson_name):
    cert_filename = 'hourofci_certificate.pdf'

    img = Image.open("../../supplementary/hci-certificate-template.jpg")
    draw = ImageDraw.Draw(img)

    cert_font   = ImageFont.truetype('../../supplementary/cruft.ttf', 150)
    cert_fontsm = ImageFont.truetype('../../supplementary/cruft.ttf', 80)
    
    _,_,w,h = cert_font.getbbox(learner_name)  
    draw.text( xy = (1650-w/2,1100-h/2), text = learner_name, fill=(0,0,0),font=cert_font)
    
    _,_,w,h = cert_fontsm.getbbox(lesson_name)
    draw.text( xy = (1650-w/2,1100-h/2 + 750), text = lesson_name, fill=(0,0,0),font=cert_fontsm)
    
    img.save(cert_filename, "PDF", resolution=100.0)   
    return cert_filename


interact_cert=interact.options(manual=True, manual_name="Create Certificate")

@interact_cert(name="Your Name")
def f(name):
    print("Congratulations",name)
    filename = make_cert(name, 'Intermediate Spatial Thinking')
    print("Download your certificate by clicking the link below.")
    
    
    


<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="hourofci_certificate.pdf?download=1" download="hourofci_certificate.pdf">Download your certificate</a></font>

